In [54]:
#import pyodbc library, used for read sql accesss files
#import pandas library, used for read excel, sql, dataframes and csv files
#import os for files uses
import pyodbc as pdb
import pandas as pd
import os





###########      MAIN VARS

planilhas = []
files = []
filterTypes = []
filterColumns = []
filterTexts = []
filterValues = []
filterInverse = []
filterIndex = 0
filterDone = False
types = ["is the exactly text", "contains the text", "#starts with text", "ends with text", "is exacly value","is above value", "is below value", "int is value"]

#options to filer:
#1=is the exactly text
#2=contains the text
#3=starts with text
#4=ends with text
#5=is exacly value
#6=is above value
#7=is below value
#8=int is value


print("You can select n numbers of filters. Let's start")


while filterDone == False:
    print("Please, choose the type of filter ({}) you want".format(filterIndex))
    print("The possible filters are: ")
    print(" 1=is the exactly text \n 2=contains the text \n 3=starts with text \n 4=ends with text \n 5=is exacly value \n 6=is above value \n 7=is below value \n 8=int is value")

    ftype = 1
    fcolumnName = ""
    ftext = ""
    fvalue = 0    
    finverse = False

    ftype = int(input("Write the number of filter ({}) type you want:".format(filterIndex)))
    if ftype < 1 or ftype > 8:
        print("invalid filter type. Please digit one of the above numbers")
    else:
        print("Now write the Column name for the filter")
        fcolumnName = input("Write column name: ")
        if fcolumnName == "":
            print("invalid column name. Please digit at least 1 character")
        else:
            valuesOk = False

            if ftype < 5:

                print("Now write the text for filter in this columns")
                ftext = input("write the text searched:")
                if ftext == "":
                    print("invalid text. Please digit at least 1 character")
                else:                    
                    valuesOk = True
            else:
                print("Now write the value for filter in this columns")
                fvalue = int(input("write the value searched:"))                
                valuesOk = True


            if valuesOk:       
                print("Now, you cant filter it inverse. Exemple: if you searched for 'a' word, if inversed, it will return only results without the 'a' world")
                fyesorno = input("You want it to be INVERSED? (y = yes | n = no)")
                if fyesorno.lower() == "s" or fyesorno.lower() == "yes":
                    finverse = True
                else:
                    finverse = False

                filterTypes.append(ftype)
                filterColumns.append(fcolumnName)
                filterTexts.append(ftext)
                filterValues.append(fvalue)
                filterInverse.append(finverse)

                print("Fine, this filter is configured. You want to add more filters? (y = yes | n = no)")
                fyesorno = input("More filters? (y = yes | n = no)")
                if fyesorno.lower() == "s" or fyesorno.lower() == "yes":
                    filterIndex += 1
                    filterDone = False
                else:                    
                    filterDone = True


print("Choosed {} filters".format(len(filterTypes)))

print("Filter: ")
print(filterTypes)
print(filterColumns)
print(filterValues)
print(filterTexts)
print(filterInverse)


#######################################




###########      LIST ALL FILES IN DIR



print("PLEASE, ENTER THE FOLDER WHERE ALL FILES ARE (can be separed by folders)")
folderPath = input("Enter the path of your file: ")
#folderPath = r"C:\Users\dougl\PythonProjects"

if os.path.exists(folderPath):
    print("STEP 1 OK: file path loaded: " + folderPath)

    for r, d, f in os.walk(folderPath):
            for file in f:
                if not "~$" in file:
                    files.append(os.path.join(r, file))
    print("STEP 2 OK: loaded all files in dir")    
else:
    print("STEP 1 ERROR: this file path doesnt exist")

#######################################

###########      SELECT ONE BY ONE AND CHECK TYPE

for file in files:   


    ###########      IF THE FILE IS FROM EXCEL (XLSX)
    if(file.endswith(".xlsx") or  file.endswith(".xls") or file.endswith(".xlsm")):

        fileName = os.path.basename(os.path.realpath(file))
        print("STEP 3...: Found a file EXCEL: {}".format(fileName)) 
        print("reading...")

        planilha = pd.read_excel(file)
        planilhas.append(planilha)

        print("STEP 3...: loaded {}".format(fileName))

    ###########      IF THE FILE IS FROM MICROSOFT ACCESS
    elif(file.endswith(".accdb")):


        fileName = os.path.basename(os.path.realpath(file))
        print("STEP 3...: Found a file ACCESSS: {}".format(fileName))         
        print("reading...")


        conn = pdb.connect(
        r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};" +
        r"Dbq={}".format(file))
        cur = conn.cursor()
        fileOnlyName = fileName.replace(".accdb", "")
        #print(fileOnlyName)
        cur.execute("SELECT * FROM {}".format(fileOnlyName))
        #print(cur)

        array = []
        index = 0

        columns = [i[0] for i in cur.description]

        while True:
            #if index > 1000:
             #   break
            row = cur.fetchone()
            if row is None:
                break
            else:
                array.append([elem for elem in row])
                index += 1

        planilha = pd.DataFrame(array, columns=columns)
        planilhas.append(planilha)
        #display(planilha)

        print("STEP 3...: loaded {}".format(fileName))

        #######################################



#######################################



###########       FILTER

    
import math  

result = []

# loop all planilhas



print(f"planilhas: {len(planilhas)}")


for planilha in planilhas:
    #print("planilha: {}".format(planilha))
    # loop all filters
    

    mask = pd.Series()
    
    print(f"filter types: {len(filterTypes)}")
    
    first = True
    
    for i in range(0, len(filterTypes)):  
        if filterColumns[i] in planilha:            
            #planilha[filterColumns[i]].fillna('0', inplace = False)
            planilha = planilha.replace("NULL", 0)
            
            if filterTypes[i] == 1:
                print("type 1")
                if first:
                    mask = (planilha[filterColumns[i]].astype(str) == filterTexts[i])
                    first = False
                else:
                    mask = (mask) & (planilha[filterColumns[i]].astype(str) == filterTexts[i])
            elif filterTypes[i] == 2:
                print("type 2")
                if first:
                    mask = (planilha[filterColumns[i]].astype(str).contains(filterTexts[i]))
                    first = False
                else:
                    mask = (mask) & (planilha[filterColumns[i]].astype(str).contains(filterTexts[i]))

            elif filterTypes[i] == 3:
                print("type 3")
                if first:
                    mask = (planilha[filterColumns[i]].astype(str).startsWith(filterTexts[i]))
                    first = False
                else:
                    mask = (mask) & (planilha[filterColumns[i]].astype(str).startsWith(filterTexts[i]))
            elif filterTypes[i] == 4:
                print("type 4")
                if first:
                    mask = (planilha[filterColumns[i]].astype(str).endsWith(filterTexts[i]))
                    first = False
                else:
                    mask = (mask) & (planilha[filterColumns[i]].astype(str).endsWith(filterTexts[i]))
            elif filterTypes[i] == 5:
                print("type 5")
                if first:
                    mask = (planilha[filterColumns[i]].astype(float) == float(filterValues[i]))
                    first = False
                else:
                    mask = (mask) & (planilha[filterColumns[i]].astype(float) == float(filterValues[i]))
            elif filterTypes[i] == 6:  
                print("type 6")          
                if first:                
                    mask = (planilha[filterColumns[i]].astype(float) >= float(filterValues[i]))
                    first = False
                else:                
                    mask = (mask) & (planilha[filterColumns[i]].astype(float) >= float(filterValues[i]))
            elif filterTypes[i] == 7:  
                print("type 7")   
                if first:
                    mask = (planilha[filterColumns[i]].astype(float) <= float(filterValues[i]))
                    first = False
                else:
                    mask = (mask) & (planilha[filterColumns[i]].astype(float) <= float(filterValues[i]))
            elif filterTypes[i] == 8:
                print("type 8")           
                if first:                
                    mask = (planilha[filterColumns[i]].astype(float) == float(filterValues[i]))
                    first = False
                else:
                    mask = (mask) & (planilha[filterColumns[i]].astype(float) == float(filterValues[i]))
            
                    
            result.append(planilha[mask])
            #print(result)

    
        
    


#######################################



###########       EXPORT

#index = 0
#for r in result:
   # r.to_excel(folderPath + r"\result" + str(index) + ".xlsx", index = False, header = True)
    #index += 1

semifinal = pd.concat(result)
final = []
total = len(semifinal.index)
max = 1000.0
if(total > max):
    for i in range(0,(math.ceil(total/max))):
        final.append(semifinal[i:((i*int(max))+int(max))])    
else:
    final.append(semifinal)

index = 0
for f in final:
    f.to_excel(folderPath + r"\result" + str(index) + ".xlsx", index = False, header = True)
    index += 1

print("DONE, EXPORTED to " + (folderPath + r"\result" + ".xlsx"))



#######################################





You can select n numbers of filters. Let's start
Please, choose the type of filter (0) you want
The possible filters are: 
 1=is the exactly text 
 2=contains the text 
 3=starts with text 
 4=ends with text 
 5=is exacly value 
 6=is above value 
 7=is below value 
 8=int is value
Write the number of filter (0) type you want:5
Now write the Column name for the filter
Write column name: CBO
Now write the value for filter in this columns
write the value searched:225125
Now, you cant filter it inverse. Exemple: if you searched for 'a' word, if inversed, it will return only results without the 'a' world
You want it to be INVERSED? (y = yes | n = no)n
Fine, this filter is configured. You want to add more filters? (y = yes | n = no)
More filters? (y = yes | n = no)n
Choosed 1 filters
Filter: 
[5]
['CBO']
[225125]
['']
[False]
PLEASE, ENTER THE FOLDER WHERE ALL FILES ARE (can be separed by folders)
STEP 1 OK: file path loaded: C:\Users\dougl\PythonProjects
STEP 2 OK: loaded all files in dir


C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\727591939.py:217: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()
C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\727591939.py:217: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()
C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\727591939.py:217: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()
C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\727591939.py:217: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()


type 5


C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\727591939.py:217: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()
C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\727591939.py:217: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()
C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\727591939.py:217: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()


filter types: 1
filter types: 1
filter types: 1
DONE, EXPORTED to C:\Users\dougl\PythonProjects\result.xlsx


planilhas: 6
filter types: 1
filter types: 1
filter types: 1


C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\2173678907.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()
C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\2173678907.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()
C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\2173678907.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()


type 5


C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\2173678907.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()
C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\2173678907.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()
C:\Users\dougl\AppData\Local\Temp\ipykernel_28156\2173678907.py:20: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  mask = pd.Series()


filter types: 1
filter types: 1
filter types: 1
DONE, EXPORTED to C:\Users\dougl\PythonProjects\result.xlsx


In [38]:
print(planilhas[2])

        CONTATOS_ID          CPF                             NOME DDD  \
0         239295776  05449407920          SILVIA DOS SANTOS RAMOS  41   
1         239296687  35421186806        GABRIELA RONDADO FERREIRA  14   
2         239296687  35421186806        GABRIELA RONDADO FERREIRA  81   
3         239297146  05994123994  MICHELLE DE LIMA VAPLAK PEREIRA  41   
4         239297416  04426060958           CARLOS MANOEL DE PAULA  41   
...             ...          ...                              ...  ..   
1543906   387513740  13214249908             RAFAEL POLATO MENDES  41   
1543907   387532921  13191809902                   DEBORAH DORTAS  41   
1543908   387590394  13314934980                    RODRIGO LOPES  41   
1543909   387590394  13314934980                    RODRIGO LOPES  41   
1543910   387607869  14325072497    RAYSSA EMYLY TENORIO FERREIRA  41   

          TELEFONE TIPO_TELEFONE SEXO             BAIRRO    CIDADE  UF  ...  \
0        995742224             3    F  CIDAD

TypeError: object of type 'builtin_function_or_method' has no len()

In [16]:
planilhas[2] = planilhas[2].replace("NULL", 0)
mask = (planilhas[2]["CBO"].astype(float) == 225125)
result = planilhas[2][mask]
print(result)

        CONTATOS_ID          CPF                                 NOME DDD  \
1624       61181664  02209435919                VITOR EMANUEL MARTINS  41   
1625       61181664  02209435919                VITOR EMANUEL MARTINS  41   
1809       61268140  07319810735              WERVERSON DO NASCIMENTO  21   
1810       61268140  07319810735              WERVERSON DO NASCIMENTO  96   
3747       70042430  04169755936                 FRANCISCO LUIZ SLOMP  45   
...             ...          ...                                  ...  ..   
1488121   369232981  70607325194  CAROLINA GERTRUDIS PLASENCIA ASOREY  41   
1501579   365206909  06581195162           DIANA MARCELA RIANO TORRES  41   
1504390   369232981  70607325194  CAROLINA GERTRUDIS PLASENCIA ASOREY  41   
1526262   353683540  01235157938     STEPHANIA TERESA MERCADO BASCOPE  48   
1533879   365206909  06581195162           DIANA MARCELA RIANO TORRES  41   

          TELEFONE TIPO_TELEFONE SEXO       BAIRRO    CIDADE  UF  ...  \
16

In [13]:
print(planilhas)

[Empty DataFrame
Columns: []
Index: [],         DDD   TELEFONE      CIDADE
0        11   70007215  RIO BRANCO
1        11   77181853  RIO BRANCO
2        11   77222741  RIO BRANCO
3        11   77270916  RIO BRANCO
4        11   77331038  RIO BRANCO
...     ...        ...         ...
523663   99  999971818  RIO BRANCO
523664   99  999972974  RIO BRANCO
523665   99  999979575  RIO BRANCO
523666   99  999980738  RIO BRANCO
523667   99  999999346  RIO BRANCO

[523668 rows x 3 columns],         CONTATOS_ID          CPF                             NOME DDD  \
0         239295776  05449407920          SILVIA DOS SANTOS RAMOS  41   
1         239296687  35421186806        GABRIELA RONDADO FERREIRA  14   
2         239296687  35421186806        GABRIELA RONDADO FERREIRA  81   
3         239297146  05994123994  MICHELLE DE LIMA VAPLAK PEREIRA  41   
4         239297416  04426060958           CARLOS MANOEL DE PAULA  41   
...             ...          ...                              ...  ..   
1

In [44]:
import math  
qtd = 1260
max = 1000
print(math.ceil(qtd / max))

2
